In [ ]:
import sys
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from pathlib import Path
from scipy.signal import lfilter, firwin

sys.path.append('../../../src/')

from utils import breathing_signal as lib

from bonsai import load_bonsai_config
load_bonsai_config(r"C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\aind-vr-foraging\Bonsai")

import harp
import harp.processing
import data_io

sns.set_context("talk")

In [ ]:
path = r"Z:\scratch\vr-foraging\690164\20231204T102852"
animal = path[23:29]
session = path[23:29]+'_'+path[30:38]
session_path = Path(path)
# session_path = Path(r"Z:\672103\20231010T104908")

# Harp Devices:
HarpBehavior = harp.HarpDevice("Behavior")
# Olfactometer = harp.HarpDevice("Olfactometer")
AnalogInput = harp.HarpDevice("AnalogInput")

harp_behavior_data = data_io.HarpSource(device=HarpBehavior, path=session_path / "Behavior.harp", name="behavior", autoload=False)

# Software Events
software_events = data_io.SoftwareEventSource(path=session_path / "SoftwareEvents", name="software_events")
active_site = software_events.streams.ActiveSite.data
active_patch = software_events.streams.ActivePatch.data
# active_patch = software_events.streams.OdorControl.data
stopped = software_events.streams.GiveReward.data


In [ ]:
encoder_data = harp.read_harp_bin(harp_behavior_data.streams.AnalogData.path)

In [ ]:
thermistordata = encoder_data[0]
thermistordata.index-=thermistordata.index[0]

In [ ]:
thermistordata = thermistordata.reset_index()
thermistordata.rename(columns={0:'data', 'Seconds': 'seconds'}, inplace=True)

In [ ]:
# Apply moving average smoothing
smoothed_ther = lib.moving_average(thermistordata['data'], window_size=5)

# Remove the 60Hz noise from the thermistor signal
smoothed_ther = lib.apply_filter(smoothed_ther, f_notch = 60, Q= 200)

# Remove slow oscillations
slow_ther = lib.moving_average(smoothed_ther, window_size=400)
new_ther = smoothed_ther/slow_ther
thermistordata['smoothed'] = new_ther

In [ ]:
thermistordata.data/=thermistordata.data.mean()

In [ ]:
for min_x, max_x in zip(range(100,150,2),range(102,152,2)):
    print(min_x, max_x)
    plt.figure(figsize=(8,4))
    sns.lineplot(x='seconds',y='data', data=thermistordata.loc[thermistordata.seconds.between(min_x,max_x)], color='k', alpha=0.5, linewidth=1, label = 'Raw signal')
    sns.lineplot(x='seconds',y='smoothed', data=thermistordata.loc[thermistordata.seconds.between(min_x,max_x)], color='red', alpha=0.5, linewidth=1, label = 'Corrected signal')
    # plt.ylim(950, 1150)
    plt.title('Thermistor signal animal: '+animal)
    plt.locator_params(axis='x', nbins=5)
    sns.despine()
    plt.show()
    plt.close()